In [1]:
# Adapted from langchain-ai Agent Supervisor
# https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb
from IPython.display import Image
image_url = "https://raw.githubusercontent.com/langchain-ai/langgraph/17f1a05b6b9e32e4e560c270d0a94246b69f1e45/examples/multi_agent/img/supervisor-diagram.png"
Image(url=image_url)

In [2]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langchain_experimental langsmith pandas langgraph

In [3]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['TAVILY_API_KEY'] = ''

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = ''
os.environ["LANGCHAIN_PROJECT"] = "Agent Supervisor"
os.environ['LANGCHAIN_API_KEY'] = ''

## Create tools

For this example, you will make an agent to do web research with a search engine, and one agent to create plots. Define the tools they'll use below:

In [4]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

## Helper Utilities

Define a helper function below, which make it easier to add new agent worker nodes.

In [5]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

We can also define a function that we will use to be the nodes in the graph - it takes care of converting the agent response to a human message. This is important because that is how we will add it the global state of the graph

In [6]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Create Agent Supervisor

It will use function calling to choose the next worker node OR finish processing.

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4o-2024-05-13")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

## Construct Graph

We're ready to start building the graph. Below, define the state and worker nodes using the function we just defined.

In [8]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You may generate safe python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

Now connect all the edges in the graph.

In [9]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

## Invoke the team

With the graph created, we can now invoke it and see how it performs!

In [12]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Code for drug discovery generative ai cancer research and print it to the terminal")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='Here are some key resources and information on the use of generative AI in drug discovery, particularly for cancer research:\n\n1. **Nature Review on AI in Cancer Research**:\n    - A comprehensive review focusing on AI applications in cancer research, including image analysis, NLP, and drug discovery.\n    - [Link to article](https://www.nature.com/articles/s41568-024-00694-7)\n\n2. **Generative Artificial Intelligence in Drug Discovery**:\n    - Discusses the basic framework, recent advances, challenges, and opportunities of generative AI in drug discovery.\n    - [Link to article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10879372/)\n\n3. **AI-Driven Generative Models in Drug Discovery**:\n    - Highlights the use of AI-driven generative models like RNNs, GANs, and VAEs in creating innovative compounds and exploring chemical space.\n    - [Link to article](https://www.tandfonline.com/do

In [13]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a research report on this website https://www.chemicalqdevice.com/.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content="## Research Report: ChemicalQDevice\n\n### Overview\nChemicalQDevice is a technology company focused on leveraging generative artificial intelligence (GenAI) and quantum computing to address some of the world's critical challenges, particularly in the medical and drug discovery sectors. The company integrates advanced AI models and quantum technologies to enhance various applications, including critical care medical devices and drug discovery for diseases like cancer and Alzheimer's.\n\n### Key Areas of Focus\n\n1. **Generative Artificial Intelligence (GenAI)**\n   - The company emphasizes the use of generative AI to solve complex problems. This includes the development of local generative AI model frameworks compatible with macOS and iOS.\n   - They explore how to develop APIs for GenAI and utilize tools like LangChain for drug discovery.\n\n2. **Quantum Computing Technologies**\n   - Quantu